# INSTACART MARKET BASKET ANALYSIS

This notebook is Part 3 of the workflow. 
Files are saved in SQL: `'./all/instadb.db'`

Business Problem
* Solution for product reorder - we will be working on this first)

Two main approaches in answering the challenge:
1. Item - item collaborative filtering (Notebook Part 2)

**2. Association Rules Mining with Apriori Algorithm **

Apriori is an algorithm used to identify frequent item sets (in our case, item pairs). It does so using a "bottom up" approach, first identifying individual items that satisfy a minimum occurence threshold. It then extends the item set, adding one item at a time and checking if the resulting item set still satisfies the specified threshold. The algorithm stops when there are no more items to add that meet the minimum occurrence requirement. 

Once the item sets have been generated using apriori, we can start mining association rules. Given that we are only looking at item sets of size 2, the association rules we will generate will be of the form {A} -> {B}. One common application of these rules is in the domain of recommender systems, where customers who purchased item A are recommended item B. Metric as the followings:
* Support - This is the percentage of orders that contains the item set. For example if there are 5 orders in total and the pairs occurs in 3 of them, support is 60%
* Confidence - Given two items, A and B, confidence measures the percentage of times that item B is purchased, given that item A was purchased. 
* Lift - Given two items, A and B, lift indicates whether there is a relationship between A and B, or whether the two items are occuring together in the same orders simply by chance (ie: at random). Unlike


In [1]:
############# IMPORT MODULES & PACKAGES ###############

import numpy as np
import scipy.stats as stats
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import random
import time

from itertools import combinations, groupby
from collections import Counter

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#SQL
from pandas.io import sql
import sqlite3
sqlite_db = './all/instadb.db'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()


### Helper Function to loop through product id return product name###
def find_product_name (data, dataset_with_names):
    product_names = [] 
    for i in data:
        list_product_names = dataset_with_names[dataset_with_names.product_id == i]['product_name'].values[0]
        product_names.append(list_product_names)        
    return product_names

### Load data

In [2]:
orders_prior_df = pd.read_sql('SELECT * FROM prior_subset', con = conn)

In [3]:
orders_prior_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,1374495,3,prior,1,1,14,,9387,1,0,Granny Smith Apples,24,4
1,1374495,3,prior,1,1,14,,17668,2,0,Unsweetened Chocolate Almond Breeze Almond Milk,91,16
2,1374495,3,prior,1,1,14,,15143,3,0,Blueberry Pint,24,4
3,1374495,3,prior,1,1,14,,16797,4,0,Strawberries,24,4
4,1374495,3,prior,1,1,14,,39190,5,0,Vanilla Unsweetened Almond Milk,91,16


In [52]:
orders_prior_df.product_name.nunique()

43282

In [4]:
orders_prior = orders_prior_df.loc[:, ['order_id', 'product_id']]

In [5]:
# Convert from DataFrame to a Series, with order_id as index and item_id as value
orders = orders_prior.set_index('order_id')['product_id'].rename('item_id')
display(orders.head(10))
print (orders.shape)
type(orders)

order_id
1374495     9387
1374495    17668
1374495    15143
1374495    16797
1374495    39190
1374495    47766
1374495    21903
1374495    39922
1374495    24810
1374495    32402
Name: item_id, dtype: int64

(5044861,)


pandas.core.series.Series

In [6]:
#  frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")
    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))

# Calculate item frequency and support
item_stats = freq(orders).to_frame("freq")
item_stats['support'] = item_stats['freq'] / order_count(orders) * 100

In [7]:
item_stats.describe()

,freq,support
count,43282.000000,43282.000000
mean,116.557946,0.023446
std,820.545266,0.165056
min,1.000000,0.000201
25%,3.000000,0.000603
50%,12.000000,0.002414
75%,50.000000,0.010058
max,75871.000000,15.261741


In [8]:
 # Filter from order_item items below min support 
min_support = 0.01
qualifying_items = item_stats[item_stats['support'] >= min_support].index
order_item = orders[orders.isin(qualifying_items)]

print("Items with support >= {}: {}".format(min_support, len(qualifying_items)))
print("Remaining order_item: {}".format(len(order_item)))

Items with support >= 0.01: 10873
Remaining order_item: 4676882


In [11]:
# Filter from order_item orders with less than 2 items
order_size = freq(order_item.index)
qualifying_orders = order_size[order_size >= 2].index
order_items = order_item[order_item.index.isin(qualifying_orders)]

print("Remaining orders with 2+ items: {}".format(len(qualifying_orders)))
print("Remaining order_item: {}".format(len(order_items)))

Remaining orders with 2+ items: 468608
Remaining order_item: 4650980


In [12]:
order_size[order_size >= 2]

1374495    10
444309      9
3002854     6
2037211     5
2710558    11
1972919     8
1839752     9
3225766     8
3160850     5
676467      6
521107      5
1402502     6
2300537     6
144358      6
1219654     2
3393222     5
2327987     2
2455079     3
634494     13
1014994     2
3227049     3
2780464     9
194741      4
1766930     5
3090814     4
2647850     3
2150649    12
695934      5
638062      6
2135842     3
           ..
578075     13
3126862    13
189130      5
678906      7
1684468     8
2408838    13
2012736    19
2036438    17
1789214     7
294807     14
2752192    15
2742017    14
1837366    19
3245259     5
1541564     7
1667339     5
1587713     5
308053      4
1821975     8
3357192     4
2350550     9
818729     11
2420455     5
1059192    18
605202     17
2547959     5
1046357     4
584166     22
1125912    18
3103285     6
Name: freq, Length: 468608, dtype: int64

In [13]:
# Recalculate item frequency and support
item_stats = freq(order_items).to_frame("freq")
item_stats['support'] = item_stats['freq'] / order_count(order_items) * 100

In [14]:
item_stats

,freq,support
24852,75585,16.129686
13176,62668,13.373225
21137,42862,9.146664
21903,37679,8.040622
47209,34817,7.429877
47766,26199,5.590814
27966,23398,4.993086
47626,23256,4.962783
27845,22977,4.903245
16797,22002,4.695182


In [15]:
# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().as_matrix()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair
            
# Get item pairs generator
item_pair_gen = get_item_pairs(order_items)

In [16]:
# Calculate item pair frequency and support
item_pairs = freq(item_pair_gen).to_frame("freqAB")
item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

print("Item pairs: {}".format(len(item_pairs)))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Item pairs: 9830778


In [17]:
# Filter from item_pairs those below min support
item_pairs = item_pairs[item_pairs['supportAB'] >= min_support]

print("Item pairs with support >= {}: {}\n".format(min_support, len(item_pairs)))

Item pairs with support >= 0.01: 54063



In [18]:
# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))

In [19]:
# Create table of association rules and compute relevant metrics
item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
item_pairs = merge_item_stats(item_pairs, item_stats)
    
item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
item_pairs['lift'] = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
   
# Association rules sorted by lift in descending order
rules = item_pairs.sort_values('lift', ascending=False)

In [20]:
rules

,item_A,item_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
53976,17163,19008,53,0.011310,146,0.031156,123,0.026248,0.363014,0.430894,13.830173
53449,30388,27553,60,0.012804,148,0.031583,161,0.034357,0.405405,0.372671,11.799765
31904,34519,11983,64,0.013657,188,0.040119,171,0.036491,0.340426,0.374269,9.329013
51355,7076,28134,51,0.010883,258,0.055057,101,0.021553,0.197674,0.504950,9.171467
14505,29126,36361,49,0.010457,217,0.046307,119,0.025394,0.225806,0.411765,8.891992
53295,13643,14962,52,0.011097,211,0.045027,130,0.027742,0.246445,0.400000,8.883564
53450,23953,27553,49,0.010457,191,0.040759,161,0.034357,0.256545,0.304348,7.467007
51503,28613,45636,51,0.010883,204,0.043533,159,0.033930,0.250000,0.320755,7.368050
47415,29479,16508,47,0.010030,159,0.033930,191,0.040759,0.295597,0.246073,7.252322
51357,7076,17766,54,0.011523,258,0.055057,142,0.030303,0.209302,0.380282,6.907095


In [24]:
#Get the names
item_name = orders_prior_df[['product_id', 'product_name']]

In [29]:
s=time.time()
item_A_name = find_product_name(rules.item_A, item_name)
print ('Item A Status: OK!')
item_B_name = find_product_name(rules.item_B, item_name)
print ('Item B Status: OK!')

rules['item_A_name'] = item_A_name
rules['item_B_name'] = item_B_name
rule_finals = rules.sort_values('lift', ascending=False)
print("Execution time:", round((time.time()-s)/60,2), "minutes")

Item A Status: OK!
Item B Status: OK!
Execution time: 14.48 minutes


In [37]:
rule_finals = rule_finals.reindex(['item_A', 'item_A_name','item_B', 'item_B_name', 'lift',
                          'freqAB','supportAB',
                          'freqA','supportA',
                          'freqB','supportB',
                          'confidenceAtoB','confidenceBtoA'], axis=1)

In [38]:
rule_finals

,item_A,item_A_name,item_B,item_B_name,lift,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA
53976,17163,Vitamin A & D Fat Free Milk,19008,Vitamin D Full Fat Milk,13.830173,53,0.011310,146,0.031156,123,0.026248,0.363014,0.430894
53449,30388,Organic Forest Berry Cream On Top Whole Milk Y...,27553,Organic Blueberry Cream On Top Whole Milk Yogurt,11.799765,60,0.012804,148,0.031583,161,0.034357,0.405405,0.372671
31904,34519,Simply 100® Blueberry Blended Non-Fat Greek Yo...,11983,Simply 100 Strawberry Blended Non-Fat Greek Yo...,9.329013,64,0.013657,188,0.040119,171,0.036491,0.340426,0.374269
51355,7076,Grain Free Chicken Formula Cat Food,28134,Grain Free Turkey Canned Cat Food,9.171467,51,0.010883,258,0.055057,101,0.021553,0.197674,0.504950
14505,29126,Organic Strawberry Chia Lowfat 2% Cottage Cheese,36361,Organic Cottage Cheese Blueberry Acai Chia,8.891992,49,0.010457,217,0.046307,119,0.025394,0.225806,0.411765
53295,13643,Zero Calorie Lemon Lime Soda,14962,Hearts of Palm,8.883564,52,0.011097,211,0.045027,130,0.027742,0.246445,0.400000
53450,23953,Cream on Top Strawberry Organic Yogurt,27553,Organic Blueberry Cream On Top Whole Milk Yogurt,7.467007,49,0.010457,191,0.040759,161,0.034357,0.256545,0.304348
51503,28613,Organic Grapefruit Ginger Sparkling Yerba Mate,45636,Cranberry Pomegranate Sparkling Yerba Mate,7.368050,51,0.010883,204,0.043533,159,0.033930,0.250000,0.320755
47415,29479,Gobble Gobble Turkey Pouch Stage 3,16508,Stage 3 Hearty Meals - Chick Chick Organic Bab...,7.252322,47,0.010030,159,0.033930,191,0.040759,0.295597,0.246073
51357,7076,Grain Free Chicken Formula Cat Food,17766,Grain Free Turkey Formula Cat Food,6.907095,54,0.011523,258,0.055057,142,0.030303,0.209302,0.380282


In [49]:
#Save to SQL 
rule_finals.to_sql(name='recommender2', if_exists = 'replace', con=conn)

### Just for fun -- Using Apyori package to test the difference

In [41]:
!pip install apyori

  Running setup.py bdist_wheel for apyori ... done
  Stored in directory: /Users/ginny/Library/Caches/pip/wheels/7b/2a/35/c0c3749c1a36d4f454ea22d8396e1b854b86340d63cbbb7949
Successfully built apyori


In [42]:
from apyori import apriori

In [47]:
s=time.time()

# Train Apriori Model
rules2 = apriori(orders.index, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2)

# Visualising the results
results = list(rules2)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 12.76 minutes


In [48]:
results

[]